In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, Xception, MobileNetV2
from PIL import Image
import os
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
drive_dir = "/content/drive/MyDrive/brain_tumor_stuff/archive/"
train_dir = drive_dir+"Training/"
test_dir = drive_dir+"Testing/"

In [4]:
train_count = 0
test_count = 0
for dir in os.listdir(train_dir):
  train_count += len(os.listdir(train_dir+dir))
for dir in os.listdir(test_dir):
  test_count += len(os.listdir(test_dir+dir))

In [5]:
print("Train count: " + str(train_count))
print("Testing count: " + str(test_count))

Train count: 2870
Testing count: 394


In [6]:
classes = {}
for i, dir in enumerate(os.listdir(train_dir)):
  classes[dir] = i

In [7]:


print(classes)

{'no_tumor': 0, 'glioma_tumor': 1, 'pituitary_tumor': 2, 'meningioma_tumor': 3}


In [8]:
def one_hot(y, n=len(classes)):
  yhot = np.zeros((n))
  yhot[y] = 1
  return yhot

In [9]:
x_train = []
y_train = []
x_test = []
y_test = []

for dir in os.listdir(train_dir):
  for file in os.listdir(train_dir+dir):
    img = Image.open(train_dir+dir+"/"+file).resize((150, 150))
    img = np.asarray(img)
    x_train.append(img)
    y_train.append(one_hot(classes[dir]))

for dir in os.listdir(test_dir):
  for file in os.listdir(test_dir+dir):
    img = Image.open(test_dir+dir+"/"+file).resize((150, 150))
    img = np.asarray(img)
    x_test.append(img)
    y_test.append(one_hot(classes[dir]))

In [10]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [11]:
print(f"x_train.shape: {x_train.shape}, y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}, y_test.shape: {y_test.shape}")

x_train.shape: (2870, 150, 150, 3), y_train.shape: (2870, 4)
x_test.shape: (394, 150, 150, 3), y_test.shape: (394, 4)


In [12]:
x_train, y_train = sklearn.utils.shuffle(x_train, y_train)

In [13]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=True)

In [14]:
print(f"x_train.shape: {x_train.shape}, y_train.shape: {y_train.shape}")
print(f"x_val.shape: {x_val.shape}, y_val.shape: {y_val.shape}")
print(f"x_test.shape: {x_test.shape}, y_test.shape: {y_test.shape}")

x_train.shape: (2870, 150, 150, 3), y_train.shape: (2870, 4)
x_val.shape: (197, 150, 150, 3), y_val.shape: (197, 4)
x_test.shape: (197, 150, 150, 3), y_test.shape: (197, 4)


In [15]:
x_train = x_train/255.0
x_val = x_val/255.0
x_test = x_test/255.0

In [16]:
callback = keras.callbacks.EarlyStopping(monitor="val_accuracy",
    min_delta=0.001,
    patience=10,
    restore_best_weights=True)

import csv

class Logger(keras.callbacks.Callback):
  def __init__(self,f='/content/drive/MyDrive/training.csv'):
    self.f = f
    with open(f, 'w', newline='') as file:
      self.writer = csv.writer(file)
  def on_epoch_end(self,epoch,logs=None):
    print(logs)

In [17]:
input_layer = VGG16(include_top=False, weights='imagenet', input_shape=[150, 150, 3])
flatten = keras.layers.Flatten()(input_layer.output)
dense1 = keras.layers.Dense(4096, "relu")(flatten)
dense2 = keras.layers.Dense(4096, "relu")(dense1)
output_layer = keras.layers.Dense(4, "softmax")(dense2)
vgg16 = keras.models.Model(input_layer.input, output_layer)
vgg16.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

58889256/58889256 [==============================] - 2s 0us/step


In [18]:
history_vgg16 = vgg16.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val), callbacks=[callback, Logger()])

Epoch 1/3
 6/90 [=>............................] - ETA: 16s - loss: 1.7790 - accuracy: 0.2760

90/90 [==============================] - 45s 314ms/step - loss: 1.1744 - accuracy: 0.4672 - val_loss: 2.0762 - val_accuracy: 0.2944
Epoch 2/3
90/90 [==============================] - 19s 214ms/step - loss: 0.7801 - accuracy: 0.6843 - val_loss: 2.1147 - val_accuracy: 0.3553
Epoch 3/3
90/90 [==============================] - 19s 216ms/step - loss: 0.5290 - accuracy: 0.7927 - val_loss: 2.7181 - val_accuracy: 0.4518


In [19]:
vgg16.evaluate(x_test, y_test)

7/7 [==============================] - 0s 57ms/step - loss: 2.7148 - accuracy: 0.4264


[2.714803457260132, 0.4263959527015686]

In [20]:
input_layer = ResNet50(include_top=False, weights='imagenet', input_shape=[150, 150, 3])
flatten = keras.layers.Flatten()(input_layer.output)
dense1 = keras.layers.Dense(4096, "relu")(flatten)
dense2 = keras.layers.Dense(4096, "relu")(dense1)
output_layer = keras.layers.Dense(4, "softmax")(dense2)
resnet = keras.models.Model(input_layer.input, output_layer)
resnet.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

94765736/94765736 [==============================] - 3s 0us/step


In [21]:
history_resnet = resnet.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val), callbacks=[callback, Logger()])

Epoch 1/3
90/90 [==============================] - 59s 302ms/step - loss: 1.1110 - accuracy: 0.8077 - val_loss: 2.2192 - val_accuracy: 0.3096
Epoch 2/3
90/90 [==============================] - 20s 226ms/step - loss: 0.0662 - accuracy: 0.9767 - val_loss: 1.9845 - val_accuracy: 0.3096
Epoch 3/3
90/90 [==============================] - 20s 226ms/step - loss: 0.0211 - accuracy: 0.9937 - val_loss: 2.5027 - val_accuracy: 0.3096


In [22]:
resnet.evaluate(x_test, y_test)

7/7 [==============================] - 0s 48ms/step - loss: 2.8843 - accuracy: 0.2335


[2.884310483932495, 0.23350253701210022]

In [23]:
input_layer = InceptionV3(include_top=False, weights='imagenet', input_shape=[150, 150, 3])
flatten = keras.layers.Flatten()(input_layer.output)
dense1 = keras.layers.Dense(4096, "relu")(flatten)
dense2 = keras.layers.Dense(4096, "relu")(dense1)
output_layer = keras.layers.Dense(4, "softmax")(dense2)
inception = keras.models.Model(input_layer.input, output_layer)
inception.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

87910968/87910968 [==============================] - 3s 0us/step


In [24]:
history_inception = inception.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val), callbacks=[callback, Logger()])

Epoch 1/3
90/90 [==============================] - 54s 222ms/step - loss: 0.6894 - accuracy: 0.7617 - val_loss: 1.9536 - val_accuracy: 0.4873
Epoch 2/3
90/90 [==============================] - 12s 136ms/step - loss: 0.1827 - accuracy: 0.9352 - val_loss: 1.2632 - val_accuracy: 0.6548
Epoch 3/3
90/90 [==============================] - 12s 135ms/step - loss: 0.0616 - accuracy: 0.9794 - val_loss: 1.5812 - val_accuracy: 0.6701


In [25]:
inception.evaluate(x_test, y_test)

7/7 [==============================] - 0s 30ms/step - loss: 1.7201 - accuracy: 0.6802


[1.7201135158538818, 0.6802030205726624]

In [26]:
input_layer = Xception(include_top=False, weights='imagenet', input_shape=[150, 150, 3])
global_avg = keras.layers.GlobalAveragePooling2D()(input_layer.output)
output_layer = keras.layers.Dense(4, "softmax")(global_avg)
xception = keras.models.Model(input_layer.input, output_layer)
xception.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

83683744/83683744 [==============================] - 3s 0us/step


In [27]:
history_xception = xception.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val), callbacks=[callback, Logger()])

Epoch 1/3
 6/90 [=>............................] - ETA: 17s - loss: 1.3261 - accuracy: 0.3958

90/90 [==============================] - 49s 271ms/step - loss: 0.5269 - accuracy: 0.8101 - val_loss: 1.3851 - val_accuracy: 0.5990
Epoch 2/3
90/90 [==============================] - 19s 213ms/step - loss: 0.0882 - accuracy: 0.9721 - val_loss: 1.1048 - val_accuracy: 0.7310
Epoch 3/3
90/90 [==============================] - 19s 208ms/step - loss: 0.0372 - accuracy: 0.9906 - val_loss: 1.1957 - val_accuracy: 0.7056


In [28]:
xception.evaluate(x_test, y_test)

7/7 [==============================] - 0s 51ms/step - loss: 1.5618 - accuracy: 0.6701


[1.5617930889129639, 0.6700507402420044]

In [29]:
mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_shape=[150, 150, 3])
flatten = keras.layers.Flatten()(input_layer.output)
dense1 = keras.layers.Dense(4096, "relu")(flatten)
dense2 = keras.layers.Dense(4096, "relu")(dense1)
output_layer = keras.layers.Dense(4, "softmax")(dense2)
mobilenet = keras.models.Model(input_layer.input, output_layer)
mobilenet.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

9406464/9406464 [==============================] - 1s 0us/step


In [30]:
history_mobilenet = mobilenet.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val), callbacks=[callback, Logger()])

Epoch 1/3
90/90 [==============================] - 46s 295ms/step - loss: 0.1515 - accuracy: 0.9679 - val_loss: 5.6502 - val_accuracy: 0.6954
Epoch 2/3
90/90 [==============================] - 24s 265ms/step - loss: 0.0889 - accuracy: 0.9829 - val_loss: 9.2975 - val_accuracy: 0.6091
Epoch 3/3
90/90 [==============================] - 23s 261ms/step - loss: 0.0431 - accuracy: 0.9871 - val_loss: 3.7038 - val_accuracy: 0.6802


In [31]:
mobilenet.evaluate(x_test, y_test)

7/7 [==============================] - 0s 54ms/step - loss: 4.0875 - accuracy: 0.6548


[4.087516784667969, 0.6548223495483398]